# Example 0

This notebook generates dark neutrino events, loads them, and describes the output formats. 

In [40]:
%load_ext autoreload
%autoreload 2

In [41]:
import os
import numpy as np
import pandas as pd

import DarkNews as dn
from DarkNews import const

## Generating events
Let us start by generating a few dark neutrino events for two cases of interest.

We do this in two different ways. 

### Command line functionality
It is possible to run the generator through the script `dn_gen`, passing the parameters as options.
The created dataset can be found in the `data/` directory tree, which is created in the same folder the script is run.

Let's try to run few points.

In [3]:
cmd_string = "dn_gen --mzprime=1.25 --m4=0.140 --neval=1000 --D_or_M=dirac --log=INFO --hepevt"
const.subprocess_cmd(cmd_string)

dn_gen --mzprime=1.25 --m4=0.140 --neval=1000 --D_or_M=dirac --log=INFO --hepevt
---------------------------------------------------------
|   ______           _        _   _                     |
|   |  _  \         | |      | \ | |                    |
|   | | | |__ _ _ __| | __   |  \| | _____      _____   |
|   | | | / _  | ___| |/ /   | .   |/ _ \ \ /\ / / __|  |
|   | |/ / (_| | |  |   <    | |\  |  __/\ V  V /\__ \  |
|   |___/ \__,_|_|  |_|\_\   \_| \_/\___| \_/\_/ |___/  |
---------------------------------------------------------

---------------------------------------------------------
Model:
	1 dirac heavy neutrino(s).
	kinetically mixed Z'
---------------------------------------------------------

---------------------------------------------------------
Experiment: 
	MiniBooNE_FHC
	fluxfile: fluxes/MiniBooNE_FHC.dat
	POT: 1.875e+21
	nuclear targets: ['C12', 'H1']
	fiducial mass: [701.1078, 116.8922] tonnes
---------------------------------------------------------

-------

Load the dataset.

In [42]:
df_d = pd.read_pickle('./data/miniboone_fhc/3plus1/m4_0.14_mzprime_1.25_dirac/pandas_df.pckl')
df_d

P_projectile                       P_target                          \
               0    1    2         3          0    1    2             3   
0       0.324029  0.0  0.0  0.324029  11.174864 -0.0 -0.0  7.099876e-14   
1       0.641148  0.0  0.0  0.641148  11.174864 -0.0 -0.0  2.831069e-14   
2       0.643916  0.0  0.0  0.643916  11.174864 -0.0 -0.0  2.264855e-14   
3       0.439218  0.0  0.0  0.439218  11.174864 -0.0 -0.0  6.594725e-14   
4       0.517441  0.0  0.0  0.517441  11.174864 -0.0 -0.0  3.119727e-14   
..           ...  ...  ...       ...        ...  ...  ...           ...   
892     0.920595  0.0  0.0  0.920595  11.174864 -0.0 -0.0 -2.331468e-15   
893     1.333730  0.0  0.0  1.333730  11.174864 -0.0 -0.0  5.551115e-15   
894     1.385862  0.0  0.0  1.385862  11.174864 -0.0 -0.0 -1.509903e-14   
895     1.804543  0.0  0.0  1.804543  11.174864 -0.0 -0.0  1.465494e-14   
896     1.084863  0.0  0.0  1.084863  11.174864 -0.0 -0.0  1.953993e-14   

      P_recoil            ... w_event_rate w_flux_avg_xsec target  \
             0         1  ...                                       
0    11.175249 -0.081689  ...   119.257414       15.109836    C12   
1    11.175428 -0.074014  ...   262.179592        4.242688    C12   
2    11.175388 -0.003709  ...   190.028187        4.537494    C12   
3    11.175118  0.067206  ...   138.683115       15.853429    C12   
4    11.174989  0.047047  ...   112.156866       18.949398    C12   
..         ...       ...  ...          ...             ...    ...   
892  11.175956 -0.153907  ...   177.250935        3.178213    C12   
893  11.175801  0.142642  ...    88.625435       12.923833    C12   
894  11.175675  0.085938  ...    60.473675       10.152689    C12   
895  11.175985  0.095496  ...     7.788518       32.538951    C12   
896  11.175662  0.081150  ...   161.228618        5.779846    C12   

    target_pdgid scattering_regime    helicity  \
                                                 
0     1000060120          coherent  conserving   
1     1000060120          coherent  conserving   
2     1000060120          coherent  conserving   
3     1000060120          coherent  conserving   
4     1000060120          coherent  conserving   
..           ...               ...         ...   
892   1000060120          coherent  conserving   
893   1000060120          coherent  conserving   
894   1000060120          coherent  conserving   
895   1000060120          coherent  conserving   
896   1000060120          coherent  conserving   

                                    underlying_process  \
                                                         
0    nu(mu) + C12 -> N4 +  C12 -> nu_light + e+ + e...   
1    nu(mu) + C12 -> N4 +  C12 -> nu_light + e+ + e...   
2    nu(mu) + C12 -> N4 +  C12 -> nu_light + e+ + e...   
3    nu(mu) + C12 -> N4 +  C12 -> nu_light + e+ + e...   
4    nu(mu) + C12 -> N4 +  C12 -> nu_light + e+ + e...   
..                                                 ...   
892  nu(mu) + C12 -> N4 +  C12 -> nu_light + e+ + e...   
893  nu(mu) + C12 -> N4 +  C12 -> nu_light + e+ + e...   
894  nu(mu) + C12 -> N4 +  C12 -> nu_light + e+ + e...   
895  nu(mu) + C12 -> N4 +  C12 -> nu_light + e+ + e...   
896  nu(mu) + C12 -> N4 +  C12 -> nu_light + e+ + e...   

                                             DATA_PATH  \
                                                         
0    ./data/miniboone_fhc/3plus1/m4_0.14_mzprime_1....   
1    ./data/miniboone_fhc/3plus1/m4_0.14_mzprime_1....   
2    ./data/miniboone_fhc/3plus1/m4_0.14_mzprime_1....   
3    ./data/miniboone_fhc/3plus1/m4_0.14_mzprime_1....   
4    ./data/miniboone_fhc/3plus1/m4_0.14_mzprime_1....   
..                                                 ...   
892  ./data/miniboone_fhc/3plus1/m4_0.14_mzprime_1....   
893  ./data/miniboone_fhc/3plus1/m4_0.14_mzprime_1....   
894  ./data/miniboone_fhc/3plus1/m4_0.14_mzprime_1....   
895  ./data/miniboone_fhc/3plus1/m4_0.14_mzprime_1....   
896  ./data/miniboone_fhc/3p

In [ ]:
cmd_string = "dn_gen --umu5=1e-3 --ud5=35 --chi=0.0031 --gD=2 --mzprime=1.25 --m4=0.080 --m5=0.140 --neval=1000 --D_or_M=majorana --log=INFO"
const.subprocess_cmd(cmd_string)

In [ ]:
df_d = pd.read_pickle('./data/miniboone_fhc/3plus2/m5_0.14_m4_0.08_mzprime_1.25_majorana/pandas_df.pckl')
df_d

### `GenLauncher` class
It is possible to run the generator through an instance of the `GenLauncher` class in the `DarkNews.GenLauncher` module.
We create an instance of this class, specifying the parameters of the run, then we use the `run()` method on this instance.
The created dataset can be found in the `data/` directory tree.
Alternatively, it an be accessed via the `df` attribute.

We run the same points as before, but with the `GenLauncher` class. When using the `run()` method it is possible to specify some parameters as `log`, `verbose` or `logfile` (which can be also specified during the construction of the object).

In [ ]:
from DarkNews.GenLauncher import GenLauncher
gen_object = GenLauncher(mzprime=1.25, m4=0.140, neval=1000, D_or_M="dirac")
gen_object.run(log="INFO")

In [ ]:
df_d = gen_object.df
df_d

In [ ]:
gen_object = GenLauncher(umu5=1e-3, ud5=35, chi=0.0031, gD=2, mzprime=1.25, m4=0.080, m5=0.140, neval=1000, D_or_M="majorana")
gen_object.run(log="INFO")

In [ ]:
df_d = gen_object.df
df_d

---
# Pandas DataFrame Output

### Metadata

Before inspecting the events inside the pandas dataframe, let us check the metadata in [df.attrs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.attrs.html). We have:

---
1. **experiment**: this is a class DarkNews.experiment.Detector, which contains all the information about the experiment for which the events are generated. It also contains a list of instances of the NuclearTarget class, which contains information about the different scattering target used.

In [ ]:
print("---------------------------------------------")
print("General description of the detector attribute")
print("---------------------------------------------")
keys = list(df_d.attrs['experiment'].__dict__.keys())
print(f"Attributes of Detector class: \n", *keys)
print(f"\nAttributes of NuclearTarget class: \n", *list(df_d.attrs['experiment'].NUCLEAR_TARGETS[0].__dict__))

In [ ]:
print("--------------------------------------------")
print("How much of each target inside the detector?")
print("--------------------------------------------")
print("\nExample of how to look up nuclear targets:")
fid_mass = df_d.attrs['experiment'].FIDUCIAL_MASS
for target, fid_mass_fraction in zip(df_d.attrs['experiment'].NUCLEAR_TARGETS, df_d.attrs['experiment'].FIDUCIAL_MASS_FRACTION_PER_TARGET):
    print("{} tonnes of {}".format(round(fid_mass_fraction*fid_mass,2), target.NAME))

---
2. **bsm_model**: this attribute contains DarkNews.model.Model, which stores all the new physics parameters used in the generation. This includes both low-level information, such as the couplings, say $g_D$, as well as high-level ones, such as the couplings of the $Z^\prime$ to electrons, $d_e^V$ and $d_e^A$.

In [ ]:
print("Some low-level parameters:\n", *list(df_d.attrs['bsm_model'].__dict__.keys())[:19])
print("\nSome high-level parameters:\n", *list(df_d.attrs['bsm_model'].__dict__.keys())[-14:])

---
### Description of the event structure
Now we look at the actual events. The DataFrame is a MultiIndex df. Each event contains the components for all the 4-momenta of the particles involved:

$$\nu _\text{P_projectile} \,+\, \text{Hadronic target} _\text{P_target} \to N _\text{P_decay_N_parent} \,+\, \text{Hadronic recoil} _\text{P_recoil}$$

$$N _\text{P_decay_N_parent} \to N^\prime _\text{P_decay_N_daughter} \,+\, \ell^+ _\text{P_decay_ell_plus} \,+\, \ell^-_\text{P_decay_ell_minus}$$

In [ ]:
df_d.dtypes

##

* w_decay_rate_0           float64 -- weight of the decay rate (sum(w) = Gamma_N)
* I_decay_rate_0           float64 -- total rate Gamma_N


* w_event_rate             float64 -- weight for the event rate (sum(w) = event rate)
* I_event_rate             float64 -- total event rate


* w_flux_avg_xsec          float64 -- weight of the flux averaged cross section (sum(w) = int(sigma*flux)* exposure)
* I_flux_avg_xsec          float64 -- int(sigma*flux)* exposure


* target                    object -- target object -- it will typically be a nucleus
* target_pdgid               int64 -- pdgID of the target 


* scattering_regime         object -- regime can be coherent or p-elastic
* helicity                  object -- helicity process, can be flipping or conserving. flipping is suppressed
* underlying_process        object -- string of the underlying process, e.g, "nu(mu) + proton_in_C12 -> N4 +  proton_in_C12 -> nu(mu) + e+ + e- + proton_in_C12"

In [ ]:
np.shape(df_d['P_decay_N_parent'].to_numpy())

In [ ]:
from DarkNews import fourvec as fv

In [ ]:
fv.df_dot4(df_d['P_decay_N_parent'],df_d['P_decay_N_parent'])